In [5]:
#Imports

import datetime
import requests
import pandas as pd
import hopsworks
import datetime
from pathlib import Path
import json
import re
import os
import warnings
warnings.filterwarnings("ignore")

In [7]:
with open('./hopsworks-api-key.txt', 'r') as file:
    os.environ["HOPSWORKS_API_KEY"] = file.read().rstrip()
    print("API Key is set")
    print("First 5 characters of API Key: ", os.environ["HOPSWORKS_API_KEY"][:5])

API Key is set
First 5 characters of API Key:  gjYGC


In [4]:
#Connect to Hopsworks to store data in the feature store



project = hopsworks.login()
fs = project.get_feature_store()

ModuleNotFoundError: Pyarrow package not found. If you want to use Apache Arrow with Hopsworks you can install the corresponding extras via `pip install "hopsworks[python]"`. You can also install pyarrow directly in your environment with `pip install pyarrow`. You will need to restart your kernel if applicable.